In [1]:
## Import packages
import torch
import time
import wandb
import numpy as np
from tqdm import tqdm

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification

In [2]:
## Set the device to train your model.
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print("Device: ", device)

Device:  mps


In [3]:
## Load the dataset
dataset = load_dataset("stanfordnlp/sst2")

## Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', clean_up_tokenization_spaces=True)

## Apply the tokenizer to the dataset.
dataset = dataset.map(lambda x: tokenizer(x['sentence'], truncation=True), batched=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [4]:
## Create batch of data using DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
## Create the model.
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Train the Model.

In [6]:
## Create a trainer class.
class CustomTrainer(Trainer):
    def _inner_training_loop(
            self,
            batch_size = None, 
            args = None,
            resume_from_checkpoint = None,
            trial = None,
            ignore_keys_for_eval = None
    ):
        number_of_epochs = args.num_train_epochs
        start_time = time.time()

        criterion = torch.nn.CrossEntropyLoss().to(device)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=1, gamma=0.9)

        train_dataloader = self.get_train_dataloader()
        eval_dataloader = self.get_eval_dataloader()

        for epoch in range(number_of_epochs):
            train_loss_per_epoch = 0
            train_acc_per_epoch = 0
            with tqdm(train_dataloader, unit = 'batch') as training_epoch:
                training_epoch.set_description(f"Training Epoch {epoch}")
                for step, inputs in enumerate(training_epoch):
                    inputs = inputs.to(device)
                    labels = inputs['labels'].to(device)

                    ## Forward pass
                    self.optimizer.zero_grad()
                    model_outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

                    ## Compute the loss
                    loss = criterion(model_outputs['logits'], labels)
                    train_loss_per_epoch += loss.item()

                    ## Calculate gradients
                    loss.backward()

                    ## Update weights
                    self.optimizer.step()
                    train_accuracy = (model_outputs['logits'].argmax(1) == labels).sum().item()
                    train_acc_per_epoch += train_accuracy

                    wandb.log({"Step Training Loss":loss.item(),
                                "Step Training Accuracy":train_acc_per_epoch,
                                "Step":step})

            ## Change the learning rate.
            self.scheduler.step()

            ## Compute the average loss and accuracy over all of the batches.
            train_loss_per_epoch /= len(train_dataloader)
            train_acc_per_epoch /= (len(train_dataloader) * batch_size)

            wandb.log({"Epoch Train Loss":train_loss_per_epoch,
                        "Epoch Train Accuracy":train_acc_per_epoch,
                        "Epoch":epoch})

            ## Run the Model on Evaluation Dataset
            eval_loss_per_epoch = 0
            eval_acc_per_epoch = 0
            with tqdm(eval_dataloader, unit='batch') as eval_epoch:
                eval_epoch.set_description(f"Evaluation Epoch {epoch}")
                with torch.no_grad():
                    for step, inputs in enumerate(eval_epoch):
                        inputs = inputs.to(device)
                        labels = inputs['labels'].to(device)

                        ## Foward pass
                        model_outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
                        
                        ## Compute loss
                        loss = criterion(model_outputs['logits'], labels)
                        eval_loss_per_epoch += loss.item()

                        ## Compute accuracy
                        eval_accuracy = (model_outputs['logits'].argmax(1) == labels).sum().item()
                        eval_acc_per_epoch += eval_accuracy
            
            eval_loss_per_epoch /= len(eval_dataloader)
            eval_acc_per_epoch /= (len(eval_dataloader) * batch_size)

            wandb.log({"Eval Loss": eval_loss_per_epoch, 
                        "Eval Accuracy": eval_acc_per_epoch})

            print(f'\tTrain Loss: {train_loss_per_epoch :.3f} | Train Acc: {train_acc_per_epoch*100:.2f}%')
            print(f'\tEval Loss: {eval_loss_per_epoch :.3f} | Eval Acc: {eval_acc_per_epoch*100:.2f}%')
        print(f'Time: {(time.time()-start_time)/60:.3f} minutes ')

In [7]:
## Set wandb parallelist to false.
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['WANDB_NOTEBOOK_NAME'] = 'hw2.ipynb'

## WandB setup to get plots and metrics.
config = dict(
    epochs = 5,
    classes = 2,
    batch_size = 64,
    learning_rate = 2e-5,
    dataset = 'sst2',
    architecture = 'bert'
)

## Setting up Training Pipeline inside WandB.
with wandb.init(project='NPL HW2', name='First Run', config=config):

    ## Define Training Arguments.
    training_args = TrainingArguments(
        output_dir='./results',
        learning_rate=wandb.config.learning_rate,
        num_train_epochs=wandb.config.epochs,
        per_device_train_batch_size=wandb.config.batch_size,
        per_device_eval_batch_size=wandb.config.batch_size
    )

    ## Initialize the trainer
    trainer = CustomTrainer(
        model=model,
        args = training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['validation'],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    ## Train the model
    trainer.train()

wandb: Currently logged in as: mohitydv09 (mohitydv09-university-of-minnesota5275). Use `wandb login --relogin` to force relogin


Evaluation Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  6.18batch/s]


	Train Loss: 0.667 | Train Acc: 100.00%
	Eval Loss: 0.607 | Eval Acc: 50.00%


Evaluation Epoch 1: 100%|██████████| 1/1 [00:00<00:00, 41.25batch/s]


	Train Loss: 0.461 | Train Acc: 100.00%
	Eval Loss: 0.600 | Eval Acc: 50.00%


...

Evaluation Epoch 18: 100%|██████████| 1/1 [00:00<00:00, 39.65batch/s]


	Train Loss: 0.091 | Train Acc: 100.00%
	Eval Loss: 0.720 | Eval Acc: 50.00%


Evaluation Epoch 19: 100%|██████████| 1/1 [00:00<00:00, 38.40batch/s]


	Train Loss: 0.088 | Train Acc: 100.00%
	Eval Loss: 0.721 | Eval Acc: 50.00%
Time: 0.115 minutes 


Epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
Epoch Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch Train Loss,█▆▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Eval Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Eval Loss,▁▁▂▂▃▄▅▅▅▆▆▇▇▇▇█████
Step,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Step Training Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Step Training Loss,█▆▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Epoch,19
Epoch Train Accuracy,1.0
Epoch Train Loss,0.08842
